In [9]:
import numpy as np
import random
import matplotlib.pyplot as plt
import time
import networkx as nx
import sys
import os
import copy
import datetime
import json
from joblib import Parallel, delayed
import CSD

import importlib
importlib.reload(CSD)
#%matplotlib inline

<module 'CSD' from 'd:\\tsubasa\\zikken\\CSD.py'>

# Setting

In [10]:
num_node_array = [25]
num_depot_array = [8]
num_driver_array = [128, 256, 512]

net_per_numnode = 2
times_per_net = 2

logit_param_q = 5 # 1 done
logit_param_n = 5

# Generate Data

In [11]:
for num_node in num_node_array:
    
    for num_depot in num_depot_array:

        for num_per_driver in num_driver_array:
            
            num_sum_driver = num_per_driver*num_node*num_node

            data = CSD.DataMaker(num_node, num_depot, num_sum_driver, logit_param_q, logit_param_n)

            for netnum in range(net_per_numnode): #net_numberは0以上net_per_numnode未満
                #inputフォルダの作成
                path_net = r"data/nn_{nn}/nv_{nv}/nd_{nd}/net_{net}/network_data".format(nn=num_node, nv=num_per_driver, nd=num_depot, net=netnum)
                # path_dep = r"data/nn_{nn}/nv_{nv}/nd_{nd}/net_{net}/input".format(nn=num_node, nv=num_per_driver, nd=num_depot, net=netnum)
                # path_dist = r"data/nn_{nn}/nv_{nv}/nd_{nd}/net_{net}/input/".format(nv=num_per_driver, nn=num_node, nd=num_depot, net=netnum)
                # path_cost = r"input/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/cost".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum)
                # path_cost_atom = r"input/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/cost_atomic".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum)
                # path_demand = r"input/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/demand".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum)
                os.makedirs(path_net, exist_ok=True)
                # os.makedirs(path_dep, exist_ok=True)
                # os.makedirs(path_dist, exist_ok=True)
                # os.makedirs(path_cost, exist_ok=True)
                # os.makedirs(path_cost_atom, exist_ok=True)
                # os.makedirs(path_demand, exist_ok=True)

                #ネットワークの作成
                data.make_network()
                data.save_network(path_net + r"/network.json")

                #各起終点の距離(dijkstra法を用いて)行列を作成
                data.make_distmat()
                data.save_distmat(path_net + r"/dist_matrix.csv")


                for exp_num in range(times_per_net):

                    path_exp = r"data/nn_{nn}/nv_{nv}/nd_{nd}/net_{net}/exp_{exp}/input".format(nn=num_node, nv=num_per_driver, nd=num_depot, net=netnum, exp=exp_num)
                    os.makedirs(path_exp, exist_ok=True)

                    #タスク起点のリストを作成
                    data.make_depot_list()
                    data.save_depot_list(path_exp + r"/depots_index.csv")
                    
                    #ドライバーのタスク実行コスト行列を作成
                    data.make_costmat()
                    data.save_costmat(path_exp + r"/cost_matrix.csv")

                    #各ODペアのドライバー数，タスク数をランダムで生成
                    data.make_demand_supply()
                    data.save_demand_supply(path_exp + r"/num_drivers.csv", path_exp + r"/num_shippers.csv")
                    
                    path_atomic = r"data/nn_{nn}/nv_{nv}/nd_{nd}/net_{net}/exp_{exp}/input/cost_atomic".format(nn=num_node, nv=num_per_driver, nd=num_depot, net=netnum, exp=exp_num)
                    os.makedirs(path_atomic, exist_ok=True)

                    #個人の費用行列を生成
                    data.make_atomic_costmat()
                    data.save_atomic_costmat(path_atomic + r"/driver_cost.csv".format(nn=num_node, nd=num_depot, net=netnum, exp=exp_num),path_atomic + r"/shipper_cost.csv".format(nn=num_node, nd=num_depot, net=netnum, exp=exp_num))

                    comp = "COMPLETE  num_driver:{nv}, num_node:{nn}, num_depot:{nd}, network_num:{net}, experience_num:{exp}\n".format(nv=num_per_driver, nn=num_node, nd=num_depot, net=netnum, exp=exp_num)
                    print(comp)
                
                print("\n")
            print("\n")
        print("\n")
    print("\n")
print("END")


completed making a network
completed making the distance matrix
completed making o-r-s-d costs
COMPLETE  num_driver:128, num_node:25, num_depot:8, network_num:0, experience_num:0

completed making o-r-s-d costs
COMPLETE  num_driver:128, num_node:25, num_depot:8, network_num:0, experience_num:1



completed making a network
completed making the distance matrix
completed making o-r-s-d costs
COMPLETE  num_driver:128, num_node:25, num_depot:8, network_num:1, experience_num:0

completed making o-r-s-d costs
COMPLETE  num_driver:128, num_node:25, num_depot:8, network_num:1, experience_num:1





completed making a network
completed making the distance matrix
completed making o-r-s-d costs
COMPLETE  num_driver:256, num_node:25, num_depot:8, network_num:0, experience_num:0

completed making o-r-s-d costs
COMPLETE  num_driver:256, num_node:25, num_depot:8, network_num:0, experience_num:1



completed making a network
completed making the distance matrix
completed making o-r-s-d costs
COMPLETE 

# Run Accel Gradient

In [12]:
for num_node in num_node_array:
    
    for num_depot in num_depot_array:

        for num_per_driver in num_driver_array:
            
            num_sum_driver = num_per_driver*num_node*num_node
            

            data_array = [[CSD.DataMaker(num_node, num_depot, num_sum_driver, logit_param_q, logit_param_n) for i in range(times_per_net)] for j in range(net_per_numnode)]
            csd_array = [[None for i in range(times_per_net)] for j in range(net_per_numnode)]

            accel_time_array = np.zeros([net_per_numnode, times_per_net])
            accel_obj_array = np.zeros([net_per_numnode, times_per_net])
            #accel_time_array = [[None for i in range(times_per_net)] for j in range(net_per_numnode)]
            ##LP_max_time_array = [[None for i in range(times_per_net)] for j in range(net_per_numnode)]
            ##LP_sum_time_array = [[None for i in range(times_per_net)] for j in range(net_per_numnode)]
            ##total_time_array = [[None for i in range(times_per_net)] for j in range(net_per_numnode)]
            ##parallel_time_array = [[None for i in range(times_per_net)] for j in range(net_per_numnode)]
            # accel_obj_array = [[None for i in range(times_per_net)] for j in range(net_per_numnode)]
            ##final_obj_array = [[None for i in range(times_per_net)] for j in range(net_per_numnode)]

            # path_output = r"data/nn_{nn}/nv_{nv}/nd_{nd}/output/proposed/python".format(nv=num_sum_driver, nn=num_node, nd=num_depot)
            path_summary_time = r"data/nn_{nn}/nv_{nv}/nd_{nd}/time_summary/proposed/accel/python".format(nv=num_per_driver, nn=num_node, nd=num_depot)
            path_summary_obj = r"data/nn_{nn}/nv_{nv}/nd_{nd}/obj_summary/proposed/accel/python".format(nv=num_per_driver, nn=num_node, nd=num_depot)

            # os.makedirs(path_output, exist_ok=True)
            os.makedirs(path_summary_time, exist_ok=True)
            os.makedirs(path_summary_obj, exist_ok=True)

            for netnum in range(net_per_numnode):


                #各ノード間のコスト行列を取得
                distmat = np.loadtxt(r"data/nn_{nn}/nv_{nv}/nd_{nd}/net_{net}/network_data/dist_matrix.csv".format(nn=num_node, nv=num_per_driver, nd=num_depot, net=netnum), delimiter=",")
                # print(distmat)

                for expnum in range(times_per_net):

                    path_middle = r"data/nn_{nn}/nv_{nv}/nd_{nd}/net_{net}/exp_{exp}/output/proposed/python/middle".format(nn=num_node, nv=num_per_driver, nd=num_depot, net=netnum, exp=expnum)
                    os.makedirs(path_middle, exist_ok=True)
                    path_time = r"data/nn_{nn}/nv_{nv}/nd_{nd}/net_{net}/exp_{exp}/output/proposed/python/time".format(nn=num_node, nv=num_per_driver, nd=num_depot, net=netnum, exp=expnum)
                    os.makedirs(path_time, exist_ok=True)

                    path_sol = r"data/nn_{nn}/nv_{nv}/nd_{nd}/net_{net}/exp_{exp}/output/proposed/python/sol".format(nn=num_node, nv=num_per_driver, nd=num_depot, net=netnum, exp=expnum)
                    os.makedirs(path_sol, exist_ok=True)

                    #タスク起点ノードの情報を入手
                    deplist = np.loadtxt(r"data/nn_{nn}/nv_{nv}/nd_{nd}/net_{net}/exp_{exp}/input/depots_index.csv".format(nn = num_node, nv=num_per_driver, nd=num_depot, net=netnum, exp = expnum), delimiter=",")
                    # print(deplist)

                    path_alloc = r"data/nn_{nn}/nv_{nv}/nd_{nd}/net_{net}/exp_{exp}/output/proposed/python/middle/real_allocation".format(nv=num_per_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum)
                    path_alloc_int = r"data/nn_{nn}/nv_{nv}/nd_{nd}/net_{net}/exp_{exp}/output/proposed/python/middle/int_allocation".format(nv=num_per_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum)
                    # path_alloc_od = r"result/result_proposed/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/allocation/allocation_{exp}/allocation_od".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum)
                    # path_alloc_rs = r"result/result_proposed/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/allocation/allocation_{exp}/allocation_rs".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum)
                    # #path_time_exp = r"result/result_proposed/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/time/time_{exp}".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum)
                    # path_price = r"result/result_proposed/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/price/price_{exp}".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum)

                    os.makedirs(path_alloc, exist_ok=True)
                    os.makedirs(path_alloc_int, exist_ok=True)
                    # os.makedirs(path_alloc_od, exist_ok=True)
                    # os.makedirs(path_alloc_rs, exist_ok=True)
                    # #os.makedirs(path_time_exp, exist_ok=True)
                    # os.makedirs(path_price, exist_ok=True)

                    num_drv = np.loadtxt(r"data/nn_{nn}/nv_{nv}/nd_{nd}/net_{net}/exp_{exp}/input/num_drivers.csv".format(nv=num_per_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum), delimiter=",").reshape([-1, 1])
                    num_drv = num_drv.reshape([-1, 1])
                    # print(num_drv)
                    num_task = np.loadtxt(r"data/nn_{nn}/nv_{nv}/nd_{nd}/net_{net}/exp_{exp}/input/num_shippers.csv".format(nv=num_per_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum), delimiter=",").reshape([-1, 1])
                    num_task = num_task.reshape([-1, 1])
                    # print(num_task)

                    data_array[netnum][expnum].load_depot_list(deplist.astype(int))
                    data_array[netnum][expnum].load_distmat(distmat)
                    data_array[netnum][expnum].load_demand_supply(num_drv, num_task)


                    csd_array[netnum][expnum] = data_array[netnum][expnum].create_CSD()



                    #加速勾配法の実行
                    csd_array[netnum][expnum].solve_dual()

                    csd_array[netnum][expnum].allocate_tasks()
                    csd_array[netnum][expnum].save_real_allocation(path_alloc + "/task_allocation_to_driver.csv", path_alloc + "/num_tasks.csv")
                    # print(csd_array[netnum][expnum].real_allocation_q.shape, csd_array[netnum][expnum].real_allocation_n.shape)
                    csd_array[netnum][expnum].rounded_allocation()


                    csd_array[netnum][expnum].save_rounded_allocation(path_alloc_int+"/int_allocation.csv", path_alloc_int+"/int_task.csv")

                    np.savetxt(path_sol + r"/sol_price.csv".format(nn=num_node, nd=num_depot, net=netnum, exp=expnum), csd_array[netnum][expnum].opt_price, delimiter=",")


                    #driver_atomic_cost, task_atomic_cost, num_driver, num_task, allocation_driver, allocation_task = LP.read_data(num_sum_driver, num_node, num_depot, netnum, expnum)

                    obj = 0

                    accel_time_array[netnum][expnum] = csd_array[netnum][expnum].time_for_solve_accel
                    #LP_max_time_array[netnum][expnum] = np.amax(LP_time)
                    #LP_sum_time_array[netnum][expnum] = np.sum(LP_time)
                    #parallel_time_array[netnum][expnum] = csd_array[netnum][expnum].time_for_solve_accel + LP_max_time_array[netnum][expnum]
                    #total_time_array[netnum][expnum] = csd_array[netnum][expnum].time_for_solve_accel + LP_sum_time_array[netnum][expnum]
                    accel_obj_array[netnum][expnum] = csd_array[netnum][expnum].opt_welfare
                    #final_obj_array[netnum][expnum] = obj

                    print("complete_{nv}_{nn}_{nd}_{net}_{exp}".format(nv=num_per_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum))

                    # print(accel_obj_array)
                    # print(accel_time_array)
                    np.savetxt(path_summary_obj + r"/accel_obj.csv", accel_obj_array, delimiter=",")
                    #np.savetxt(path_obj + r"/final_obj_{nv}{nn}_{nd}.csv".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum), final_obj_array, delimiter=",")
                    np.savetxt(path_summary_time + r"/accel_time.csv", accel_time_array, delimiter=",")
                    #np.savetxt(path_time + r"/LP_max_time_{nv}_{nn}_{nd}.csv".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum), LP_max_time_array, delimiter=",")
                    #np.savetxt(path_time + r"/LP_sum_time_{nv}_{nn}_{nd}.csv".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum), LP_sum_time_array, delimiter=",")
                    #np.savetxt(path_time + r"/parallell_time_{nv}_{nn}_{nd}.csv".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum), parallel_time_array, delimiter=",")
                    #np.savetxt(path_time + r"/total_time_{nv}_{nn}_{nd}.csv".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum), total_time_array, delimiter=",")


completed loading the distance matrix
processing... 100 objective function: 1458330.2417715515 conv: 22652.964817999622 L: 4390.927778387033
processing... 200 objective function: 1459996.9335661258 conv: 1503.9016359759635 L: 4390.927778387033


d:\tsubasa\zikken\CSD.py:252: RuntimeWarning: overflow encountered in exp
  exp_cu_rs = np.exp(-logit_param_q*(cost_rs - price))
d:\tsubasa\zikken\CSD.py:256: RuntimeWarning: invalid value encountered in matmul
  sumexp_od = sumexp_os@exp_cost_sd + exp_od + 1.0e-300 # size: i*j = i*s + s*j
d:\tsubasa\zikken\CSD.py:270: RuntimeWarning: invalid value encountered in multiply
  prob_rs_o = exp_ors_o*(sumexp_os_o**-1) # size: r*s = r*s / 1*s sumが1であることを確認
d:\tsubasa\zikken\CSD.py:276: RuntimeWarning: invalid value encountered in multiply
  prob_sd_o = exp_osd_o*(sumexp_od_o**-1) # size: s*j = s*j / 1*j sumが1であることを確認
d:\tsubasa\zikken\CSD.py:286: RuntimeWarning: overflow encountered in exp
  exp_cu_rs_1 = np.exp(-logit_param_n*price)
d:\tsubasa\zikken\CSD.py:290: RuntimeWarning: invalid value encountered in multiply
  prob_rs_1 = exp_cu_rs_1*(sumexp_rs_k**-1)


processing... 300 objective function: 1460093.5334989862 conv: 59.9374516956621 L: 4390.927778387033
processing... 400 objective function: 1460099.4027529072 conv: 12.76031329370439 L: 4390.927778387033
processing... 500 objective function: 1460101.5885969682 conv: 6.089897943553216 L: 4390.927778387033
processing... 600 objective function: 1460102.3266817667 conv: 1.0854962719606402 L: 4390.927778387033
complete_128_25_8_0_0
completed loading the distance matrix
processing... 100 objective function: 1532916.7284717555 conv: 34194.95217400245 L: 2999.0627541746
processing... 200 objective function: 1533314.6638444301 conv: 197.83658405017465 L: 2999.0627541746
processing... 300 objective function: 1533321.9604611383 conv: 34.020227804918996 L: 2999.0627541746
complete_128_25_8_0_1
completed loading the distance matrix
processing... 100 objective function: 1406439.1600302514 conv: 28388.646679916328 L: 4830.020556225737
processing... 200 objective function: 1408101.9001366072 conv: 1890